![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Legal/5.Financial_RE_ZeroShotRE.ipynb)

# Financial Relation Extraction(RE) and Zero-shot Relation Extraction

## Colab Setup

In [ ]:
import json
import os
from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

# Adding license key-value pairs to environment variables

os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

## Start Spark Session

In [ ]:
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark = sparknlp_jsl.start(license_keys['SECRET'])

spark

## Extract Acquisition and Subsidiary Relationships

This is a demonstration of using SparkNLP for extracting the following relations.

- **DATE-ORG**
- **DATE-ALIAS**
- **DATE-PRODUCT**
- **ORG-ORG**

The aim of this model is to retrieve acquisition or subsidiary relationships between Organizations, included when the acquisition was carried out **was_acquired** and by whom **was_acquired_by**. Subsidiaries are tagged with the relationship **is_subsidiary_of**.

In [ ]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_detector = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "xx")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")
        
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

embeddings = BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

ner_model = FinanceNerModel.pretrained("finner_orgs_prods_alias","en","finance/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner_org")

ner_converter = NerConverter()\
    .setInputCols(["sentence","token","ner_org"])\
    .setOutputCol("ner_chunk_org")

token_classifier = DeBertaForTokenClassification.pretrained("deberta_v3_base_token_classifier_ontonotes", "en")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("ner_date")\
    .setCaseSensitive(True)\
    .setMaxSentenceLength(512) 

ner_converter_date = NerConverter()\
    .setInputCols(["sentence","token","ner_date"])\
    .setOutputCol("ner_chunk_date")\
    .setWhiteList(["DATE"])

chunk_merger = ChunkMergeApproach()\
    .setInputCols("ner_chunk_org", "ner_chunk_date")\
    .setOutputCol('ner_chunk')

re_model = RelationExtractionDLModel.pretrained("finre_acquisitions_subsidiaries", "en", "finance/models")\
    .setPredictionThreshold(0.3)\
    .setInputCols(["ner_chunk", "document"])\
    .setOutputCol("relations")

pipeline = Pipeline(stages=[
        document_assembler,
        sentence_detector,
        tokenizer,
        embeddings,
        ner_model,
        ner_converter,
        token_classifier,
        ner_converter_date,
        chunk_merger,
        re_model
        ])
empty_df = spark.createDataFrame([[""]]).toDF("text")

re_model = pipeline.fit(empty_df)

light_model = LightPipeline(re_model)


In [ ]:
ner_model.getClasses()

['O', 'B-ORG', 'I-ORG', 'B-ALIAS', 'I-ALIAS', 'I-PRODUCT', 'B-PRODUCT']

### Create Generic Function to Show Relations in Dataframe

In [ ]:
def get_relations_df (results, col='relations'):
    rel_pairs=[]
    for i in range(len(results)):
        for rel in results[i][col]:
            rel_pairs.append((
              rel.result, 
              rel.metadata['entity1'], 
              rel.metadata['entity1_begin'],
              rel.metadata['entity1_end'],
              rel.metadata['chunk1'], 
              rel.metadata['entity2'],
              rel.metadata['entity2_begin'],
              rel.metadata['entity2_end'],
              rel.metadata['chunk2'], 
              rel.metadata['confidence']
          ))
    rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])
    return rel_df

### Getting Result with Light Pipeline

LightPipelines are Spark NLP specific Pipelines, equivalent to Spark ML Pipeline, but meant to deal with smaller amounts of data. They’re useful working with small datasets, debugging results, or when running either training or prediction from an API that serves one-off requests.
Spark NLP LightPipelines are Spark ML pipelines converted into a single machine but the multi-threaded task, becoming more than 10x times faster for smaller amounts of data (small is relative, but 50k sentences are roughly a good maximum). To use them, we simply plug in a trained (fitted) pipeline and then annotate a plain text. We don't even need to convert the input text to DataFrame in order to feed it into a pipeline that's accepting DataFrame as an input in the first place. This feature would be quite useful when it comes to getting a prediction for a few lines of text from a trained ML model.

In [ ]:
sample_text = """WhatsApp was acquired by Facebook for $19 billion in 2014. Now, WhatsApp is a subsidiary of Meta."""

In [ ]:
result = light_model.fullAnnotate(sample_text)

In [ ]:
rel_df = get_relations_df(result)

rel_df

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,was_acquired_by,ORG,0,7,WhatsApp,ORG,25,32,Facebook,0.9747936
1,was_acquired,ORG,0,7,WhatsApp,DATE,53,56,2014,0.9661905
2,was_acquired,ORG,25,32,Facebook,DATE,53,56,2014,0.9053148
3,is_subsidiary_of,ORG,64,71,WhatsApp,ORG,92,95,Meta,0.99506086


### Visualization of Extracted Relations

We use **RelationExtractionVisualizer** method of **spark-nlp-display** library for visualization fo the extracted relations between the entities.

In [ ]:
from sparknlp_display import RelationExtractionVisualizer

re_vis = RelationExtractionVisualizer()

re_vis.display(result = result[0],
               relation_col = "relations",
               document_col = "document",
               show_relations=True
               )

## Relation extraction between ORGS, PRODUCTS and their ALIASES

This model shows relations between ORG (Companies), PRODUCT (Products) and their ALIAS in financial documents.

The extracted relationships are as follows:

- **ORG-ALIAS**
- **PRODUCT-ALIAS**

In [ ]:
ner_model = FinanceNerModel.pretrained("finner_orgs_prods_alias", "en", "finance/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverter()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")

pos = PerceptronModel.pretrained("pos_anc", 'en')\
    .setInputCols("sentence", "token")\
    .setOutputCol("pos")
    
dependency_parser = DependencyParserModel().pretrained("dependency_conllu", "en")\
    .setInputCols(["sentence", "pos", "token"])\
    .setOutputCol("dependencies")

re_ner_chunk_filter = RENerChunksFilter()\
    .setInputCols(["ner_chunk", "dependencies"])\
    .setOutputCol("re_ner_chunks")\
    .setRelationPairs(["ORG-ALIAS, PRODUCT-ALIAS"])

re_model = RelationExtractionDLModel.pretrained("finre_org_prod_alias", "en", "finance/models")\
    .setPredictionThreshold(0.3)\
    .setInputCols(["ner_chunk", "sentence"])\
    .setOutputCol("relations")

nlpPipeline = Pipeline(stages=[
        document_assembler,
        sentence_detector,
        tokenizer,
        embeddings,
        ner_model,
        ner_converter,
        pos,
        dependency_parser,
        re_ner_chunk_filter,
        re_model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

re_model = nlpPipeline.fit(empty_data)

light_model = LightPipeline(re_model)

### Getting Result with Light Pipeline

In [ ]:
sample_text = """On March 12, 2020 we closed a Loan and Security Agreement with Hitachi Capital America Corp. ("Hitachi") the terms of which are described in this report which replaced our credit facility with Western Alliance Bank."""

result = light_model.fullAnnotate(sample_text)

rel_df = get_relations_df(result)

rel_df[rel_df["relation"] != "no_rel"]

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,has_alias,ORG,63,91,Hitachi Capital America Corp.,ALIAS,95,101,Hitachi,0.9983972


In [ ]:
pd.DataFrame([(x.result, x.metadata["entity"]) for x in result[0]["ner_chunk"]], columns=["text", "ner"])

,text,ner
0,Hitachi Capital America Corp.,ORG
1,Hitachi,ALIAS
2,Western Alliance Bank,ORG


### Visualization of Extracted Relations

In [ ]:
from sparknlp_display import RelationExtractionVisualizer

re_vis = RelationExtractionVisualizer()

re_vis.display(result = result[0],
               relation_col = "relations",
               document_col = "document",
               exclude_relations = ["no_rel"],
               show_relations=True
               )

## Zero Shot Relation Extraction to Extract Relations Between Financial Entities

This is a Zero-shot Relation Extraction Model, meaning that it does not require any training data, just few examples of of the relations types you are looking for, to output a proper result.

**Make sure you keep the proper syntax of the relations you want to extract**

In [ ]:
ner_model = FinanceNerModel.pretrained("finner_financial_small", "en", "finance/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner")\

ner_converter = NerConverter()\
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")

re_model = ZeroShotRelationExtractionModel.pretrained("finre_zero_shot", "en", "finance/models")\
    .setInputCols(["ner_chunk", "sentence"]) \
    .setOutputCol("relations")\
    .setMultiLabel(False)

re_model.setRelationalCategories({
    "profit_decline_by": ["{PROFIT_DECLINE} decreased by {AMOUNT} from", "{PROFIT_DECLINE} decreased by {AMOUNT} to"],
    "profit_decline_by_per": ["{PROFIT_DECLINE} decreased by a {PERCENTAGE} from", "{PROFIT_DECLINE} decreased by a {PERCENTAGE} to"],
    "profit_decline_from": ["{PROFIT_DECLINE} decreased from {AMOUNT}", "{PROFIT_DECLINE} decreased from {AMOUNT} for the year"],
    "profit_decline_from_per": ["{PROFIT_DECLINE} decreased from {PERCENTAGE} to", "{PROFIT_DECLINE} decreased from {PERCENTAGE} to a total of"],
    "profit_decline_to": ["{PROFIT_DECLINE} to {AMOUNT}"],
    "profit_increase_from": ["{PROFIT_INCREASE} from {AMOUNT}"],
    "profit_increase_to": ["{PROFIT_INCREASE} to {AMOUNT}"],    
    "expense_decrease_by": ["{EXPENSE_DECREASE} decreased by {AMOUNT}"],
    "expense_decrease_by_per": ["{EXPENSE_DECREASE} decreased by a {PERCENTAGE}"],
    "expense_decrease_from": ["{EXPENSE_DECREASE} decreased from {AMOUNT}"],    
    "expense_decrease_to": ["{EXPENSE_DECREASE} for a total of {AMOUNT} for the fiscal year"],    
    "has_date": ["{AMOUNT} for the fiscal year ended {FISCAL_YEAR}", "{PERCENTAGE} for the fiscal year ended {FISCAL_YEAR}"]
})

pipeline = Pipeline(stages =[
                document_assembler,  
                sentence_detector,
                tokenizer, 
                embeddings,
                ner_model,
                ner_converter,
                re_model
               ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

light_model = LightPipeline(model)

In [ ]:
ner_model.getClasses()

['O',
 'B-PERCENTAGE',
 'B-FISCAL_YEAR',
 'I-FISCAL_YEAR',
 'B-PROFIT_INCREASE',
 'I-EXPENSE_INCREASE',
 'B-CURRENCY',
 'B-EXPENSE_INCREASE',
 'B-EXPENSE_DECREASE',
 'I-AMOUNT',
 'I-DATE',
 'I-PROFIT_INCREASE',
 'B-AMOUNT',
 'I-PROFIT_DECLINE',
 'I-CURRENCY',
 'I-EXPENSE',
 'B-DATE',
 'I-PERCENTAGE',
 'B-EXPENSE',
 'B-PROFIT_DECLINE',
 'I-PROFIT',
 'B-PROFIT',
 'I-EXPENSE_DECREASE']

In [ ]:
sample_text = """License fees revenue decreased 40 %, or $ 0.5 million to $ 0.7 million for the year ended December 31, 2020 compared to $ 1.2 million for the year ended December 31, 2019. Services revenue increased 4 %, or $ 1.1 million, to $ 25.6 million for the year ended December 31, 2020 from $ 24.5 million for the year ended December 31, 2019. Costs of revenue, excluding depreciation and amortization increased by $ 0.1 million, or 2 %, to $ 8.8 million for the year ended December 31, 2020 from $ 8.7 million for the year ended December 31, 2019.  Also, a decrease in travel costs of $ 0.4 million due to travel restrictions caused by the global pandemic. As a percentage of revenue, cost of revenue, excluding depreciation and amortization was 34 % for each of the years ended December 31, 2020 and 2019. Sales and marketing expenses decreased 20 %, or $ 1.5 million, to $ 6.0 million for the year ended December 31, 2020 from $ 7.5 million for the year ended December 31, 2019"""

data = spark.createDataFrame([[sample_text]]).toDF("text")

result = model.transform(data)


**NER output**

In [ ]:
result.selectExpr("explode(ner_chunk) as ner").show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------+
|ner                                                                                                                       |
+--------------------------------------------------------------------------------------------------------------------------+
|{chunk, 0, 19, License fees revenue, {entity -> PROFIT_DECLINE, sentence -> 0, chunk -> 0, confidence -> 0.56023335}, []} |
|{chunk, 31, 32, 40, {entity -> PERCENTAGE, sentence -> 0, chunk -> 1, confidence -> 1.0}, []}                             |
|{chunk, 40, 40, $, {entity -> CURRENCY, sentence -> 0, chunk -> 2, confidence -> 1.0}, []}                                |
|{chunk, 42, 52, 0.5 million, {entity -> AMOUNT, sentence -> 0, chunk -> 3, confidence -> 1.0}, []}                        |
|{chunk, 57, 57, $, {entity -> CURRENCY, sentence -> 0, chunk -> 4, confidence -> 1.0}, []}                                |


**Relations output**

In [ ]:
result.selectExpr("explode(relations) as relation").show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|relation                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
+-------------------------------------------------------------------------------------

### Getting Result with Light Pipeline

In [ ]:
result = light_model.fullAnnotate(sample_text)

rel_df = get_relations_df(result)

rel_df[rel_df["relation"] != "no_rel"]

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,expense_decrease_from,EXPENSE_DECREASE,799,826,Sales and marketing expenses,AMOUNT,923,933,7.5 million,0.95026535
1,has_date,AMOUNT,59,69,0.7 million,FISCAL_YEAR,90,106,"December 31, 2020",0.97494584
2,profit_increase_to,PROFIT_INCREASE,172,187,Services revenue,AMOUNT,227,238,25.6 million,0.984317
3,has_date,PERCENTAGE,31,32,40,FISCAL_YEAR,90,106,"December 31, 2020",0.89398825
4,expense_decrease_from,EXPENSE_DECREASE,561,572,travel costs,AMOUNT,579,589,0.4 million,0.985522
5,profit_increase_from,PROFIT_INCREASE,172,187,Services revenue,AMOUNT,209,219,1.1 million,0.95954156
6,has_date,AMOUNT,42,52,0.5 million,FISCAL_YEAR,90,106,"December 31, 2020",0.7735786
7,profit_increase_from,PROFIT_INCREASE,172,187,Services revenue,AMOUNT,284,295,24.5 million,0.5014099
8,has_date,PERCENTAGE,199,199,4,FISCAL_YEAR,259,275,"December 31, 2020",0.5061663
9,profit_decline_from_per,PROFIT_DECLINE,0,19,License fees revenue,PERCENTAGE,31,32,40,0.9937664


### Visualization of Extracted Relations

In [ ]:
from sparknlp_display import RelationExtractionVisualizer

re_vis = RelationExtractionVisualizer()

re_vis.display(result = result[0],
               relation_col = "relations",
               document_col = "document",
               exclude_relations = ["no_rel"],
               show_relations=True
               )